In [1]:
import numpy as np
from numpy.random import default_rng, Generator
import trimesh
from trimesh import load, Trimesh
from trimesh.primitives import Sphere
from magnet_pinn.generator.samplers import BlobSampler, TubeSampler, PropertySampler
from magnet_pinn.generator.structures import Blob, Tube
from magnet_pinn.generator.typing import StructurePhantom
from magnet_pinn.generator.transforms import ToMesh, MeshesCutout, MeshesCleaning, Compose
from magnet_pinn.generator.io import MeshWriter
from magnet_pinn.generator.structures import CustomMeshStructure
from magnet_pinn.generator.utils import generate_fibonacci_points_on_sphere

In [2]:
class MeshBlobSampler:
    
    def __init__(self, child_radius: float):
        if child_radius <= 0:
            raise ValueError("child_radius must be positive")
        self.child_radius = child_radius
    
    def _sample_inside_position(self, mesh: Trimesh, rng: Generator) -> np.ndarray:
        """Sample a single point uniformly from the mesh volume."""
        # draw one point from mesh interior using Trimesh volume sampling
        return trimesh.sample.volume_mesh(mesh, count=5)[0]
    
    def sample_children_blobs(self, parent_mesh_structure: CustomMeshStructure, 
                            num_children: int, rng: Generator, 
                            max_iterations: int = 100000) -> list[Blob]:
        if num_children == 0:
            return []
            
        mesh = parent_mesh_structure.mesh
        placed_blobs: list[Blob] = []
        
        for _ in range(num_children):
            attempts = 0
            blob_placed = False
            
            while attempts < max_iterations and not blob_placed:
                position = self._sample_inside_position(mesh, rng)
                
                if self._validates_blob_collision(position, placed_blobs):
                    blob = Blob(position, self.child_radius, 
                                  seed=rng.integers(0, 2**32-1).item())
                    placed_blobs.append(blob)
                    blob_placed = True
                
                attempts += 1
            
            if not blob_placed:
                break
            
        return placed_blobs
    
    def _validates_blob_collision(self, position: np.ndarray, existing_blobs: list[Blob]) -> bool:
        # enforce no-overlap by checking pairwise distances
        min_distance = 2 * self.child_radius  # No overlap
        # if no existing blobs, always valid
        if not existing_blobs:
            return True
        # collect existing positions plus candidate
        points = np.vstack([blob.position for blob in existing_blobs] + [position])
        # compute pairwise distances
        dists = np.linalg.norm(points[:, None, :] - points[None, :, :], axis=-1)
        # ignore self-distances
        dists += np.eye(len(points)) * 1e10
        # each point's nearest neighbor distance
        min_dists = dists.min(axis=0)
        return np.all(min_dists >= min_distance)

In [3]:
class MeshTubeSampler:
    """Sampler for tubes inside a mesh volume with collision detection and radius variation."""
    def __init__(self, tube_max_radius: float, tube_min_radius: float):
        if tube_max_radius <= 0:
            raise ValueError("tube_max_radius must be positive")
        if tube_min_radius <= 0:
            raise ValueError("tube_min_radius must be positive")
        if tube_min_radius >= tube_max_radius:
            raise ValueError("tube_min_radius must be less than tube_max_radius")
        self.tube_max_radius = tube_max_radius
        self.tube_min_radius = tube_min_radius

    def _sample_inside_position(self, mesh: Trimesh, rng: Generator) -> np.ndarray:
        """Sample a single start point uniformly from the mesh volume."""
        return trimesh.sample.volume_mesh(mesh, count=5)[0]

    def sample_tubes(self, parent_mesh_structure: CustomMeshStructure, num_tubes: int, rng: Generator,
                    max_iterations: int = 10000) -> list[Tube]:
        mesh = parent_mesh_structure.mesh
        placed_tubes: list[Tube] = []
        for i in range(num_tubes):
            attempts = 0
            tube_placed = False
            while attempts < max_iterations and not tube_placed:
                radius = rng.uniform(self.tube_min_radius, self.tube_max_radius)
                start = self._sample_inside_position(mesh, rng)
                direction = rng.normal(size=start.shape)
                # normalize direction
                direction = direction / np.linalg.norm(direction)
                tube = Tube(start, direction, radius)
                # collision check using Tube.distance_to_tube
                is_intersecting = any(
                    Tube.distance_to_tube(tube, other) < tube.radius + other.radius
                    for other in placed_tubes
                )
                if not is_intersecting:
                    placed_tubes.append(tube)
                    tube_placed = True
                attempts += 1
            if not tube_placed:
                break
        return placed_tubes



In [4]:
class CylinderPhantom:
    def __init__(self, stl_mesh_path: str, num_children_blobs: int = 3, 
                 blob_radius_decrease_per_level: float = 0.3, num_tubes: int = 5,
                 relative_tube_max_radius: float = 0.1, relative_tube_min_radius: float = 0.01):
        self.parent_structure = CustomMeshStructure(stl_mesh_path)

        child_radius = self.parent_structure.radius * blob_radius_decrease_per_level

        self.num_children_blobs = num_children_blobs
        self.num_tubes = num_tubes
        
        self.child_sampler = MeshBlobSampler(
            child_radius
        )
        
        tube_max_radius = relative_tube_max_radius * self.parent_structure.radius
        tube_min_radius = relative_tube_min_radius * self.parent_structure.radius
        self.tube_sampler = MeshTubeSampler(tube_max_radius, tube_min_radius)
    
    def _estimate_sampling_radius(self):
        return 0.7 * self.parent_structure.radius

    def generate(self, seed: int = None) -> StructurePhantom:
        rng = default_rng(seed)
        
        children_blobs = self.child_sampler.sample_children_blobs(
            self.parent_structure,
            num_children=self.num_children_blobs,
            rng=rng
        )
        
        sampling_radius = self._estimate_sampling_radius()
        tubes = self.tube_sampler.sample_tubes(
            parent_mesh_structure=self.parent_structure,
            num_tubes=self.num_tubes,
            rng=rng
        )
        
        return StructurePhantom(
            parent=self.parent_structure,
            children=children_blobs,
            tubes=tubes
        )

In [5]:
cylinder_phantom = CylinderPhantom(
    stl_mesh_path="./phantom.stl",
    num_children_blobs=5,
    blob_radius_decrease_per_level=0.2,
    num_tubes=15,
    relative_tube_max_radius=0.08,
    relative_tube_min_radius=0.02
)

print(f"Loaded mesh with {len(cylinder_phantom.parent_structure.mesh.vertices)} vertices")
print(f"Mesh bounds: {cylinder_phantom.parent_structure.mesh.bounds}")
print(f"Mesh center: {cylinder_phantom.parent_structure.position}")
print(f"Effective radius: {cylinder_phantom.parent_structure.radius:.2f}")

Loaded mesh with 100 vertices
Mesh bounds: [[-125.6217  -150.259   -138.     ]
 [ 125.6217    79.74104  138.     ]]
Mesh center: [ 5.45850530e-07 -4.16992509e+01  3.24919470e-15]
Effective radius: 190.31


In [6]:
meshes = cylinder_phantom.generate(seed=42)
print(f"Generated {len(meshes.children)} children blobs and {len(meshes.tubes)} tubes")

Generated 5 children blobs and 15 tubes


In [7]:
workflow = Compose([
    ToMesh()
])

In [8]:
resulting_meshes = workflow(meshes)

In [10]:
trimesh.boolean.union(
    resulting_meshes.children + resulting_meshes.tubes + [resulting_meshes.parent]
).show()